In [1]:
from IPython.display import Image

In [2]:
Image(url='https://miro.medium.com/v2/resize:fit:828/format:webp/1*uyuyOW1VBqmF5Gtv225XHQ.gif', width=400)

In [5]:
import numpy as np
import time
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
os.environ['http_proxy'] = 'http://127.0.0.1:7890'
os.environ['https_proxy'] = 'http://127.0.0.1:7890'

device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained("gpt2-xl")
model = AutoModelForCausalLM.from_pretrained("gpt2-xl").to(device)

for use_cache in (True, False):
    times = []
    for _ in range(10):  # measuring 10 generations
        start = time.time()
        model.generate(**tokenizer("What is KV caching?", return_tensors="pt").to(device), 
                       use_cache=use_cache, max_new_tokens=1000)
        times.append(time.time() - start)
    print(f"{'with' if use_cache else 'without'} KV caching: {round(np.mean(times), 3)} +- {round(np.std(times), 3)} seconds")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


with KV caching: 22.736 +- 0.364 seconds


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


without KV caching: 65.567 +- 0.079 seconds


In [25]:
L, l, d = 5, 2, 3
K_past = np.random.randn(L, 3)
V_past = np.random.randn(L, 3)
Q_past = np.random.randn(L, 3)

Q_new = np.random.randn(l, 3)
K_new = np.random.randn(l, 3)
V_new = np.random.randn(l, 3)


In [26]:
def create_custom_matrix(n):
    # 创建一个全为负无穷的矩阵
    matrix = np.full((n, n), -np.inf)
    
    # 将下三角部分（包括对角线）设置为0
    lower_triangle_indices = np.tril_indices(n)
    matrix[lower_triangle_indices] = 0
    
    return matrix

In [27]:
M1 = create_custom_matrix(5)

In [32]:
import scipy as sp
sp.special.softmax((Q_past.dot(K_past.T))/np.sqrt(3) + M1, axis=1)

array([[1.        , 0.        , 0.        , 0.        , 0.        ],
       [0.50531718, 0.49468282, 0.        , 0.        , 0.        ],
       [0.29997653, 0.536565  , 0.16345847, 0.        , 0.        ],
       [0.76033611, 0.0168573 , 0.14218103, 0.08062556, 0.        ],
       [0.5052315 , 0.07355313, 0.14660357, 0.16042448, 0.11418731]])

In [29]:
M2 = create_custom_matrix(7)

In [33]:
Q = np.concatenate([Q_past, Q_new], axis=0)
K = np.concatenate([K_past, K_new], axis=0)
sp.special.softmax((Q.dot(K.T))/np.sqrt(3) + M2, axis=1)

array([[1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.50531718, 0.49468282, 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.29997653, 0.536565  , 0.16345847, 0.        , 0.        ,
        0.        , 0.        ],
       [0.76033611, 0.0168573 , 0.14218103, 0.08062556, 0.        ,
        0.        , 0.        ],
       [0.5052315 , 0.07355313, 0.14660357, 0.16042448, 0.11418731,
        0.        , 0.        ],
       [0.23272304, 0.03505261, 0.17606401, 0.21343528, 0.03424012,
        0.30848495, 0.        ],
       [0.26807313, 0.12724091, 0.02830417, 0.02682431, 0.34348339,
        0.01331476, 0.19275933]])

In [34]:
import scipy as sp
sp.special.softmax((Q_past.dot(K_past.T))/np.sqrt(3), axis=1)

array([[0.38646134, 0.1143066 , 0.15626135, 0.19143388, 0.15153683],
       [0.23936974, 0.23433222, 0.06876375, 0.10532107, 0.35221322],
       [0.15590318, 0.27886245, 0.0849523 , 0.06249587, 0.41778621],
       [0.73318379, 0.01625531, 0.13710361, 0.07774635, 0.03571094],
       [0.5052315 , 0.07355313, 0.14660357, 0.16042448, 0.11418731]])

In [35]:
sp.special.softmax((Q.dot(K.T))/np.sqrt(3), axis=1)

array([[0.27338655, 0.08086162, 0.11054082, 0.13542221, 0.10719864,
        0.12119723, 0.17139293],
       [0.18972051, 0.18572786, 0.05450102, 0.08347575, 0.2791584 ,
        0.04001717, 0.1673993 ],
       [0.12577075, 0.22496488, 0.06853301, 0.05041689, 0.33703794,
        0.03791266, 0.15536387],
       [0.53206941, 0.01179643, 0.0994957 , 0.0564203 , 0.02591533,
        0.11822246, 0.15608036],
       [0.35676099, 0.05193834, 0.10352172, 0.11328113, 0.08063151,
        0.11579268, 0.17807362],
       [0.20771485, 0.03128589, 0.15714434, 0.19049973, 0.03056071,
        0.27533546, 0.10745902],
       [0.26807313, 0.12724091, 0.02830417, 0.02682431, 0.34348339,
        0.01331476, 0.19275933]])

## encoder-decoder vs. decoder only 

- 以多轮对话为例，从计算复杂度的角度探索为什么 decoder-only 更优
- 定义
    - $L$: past sequence length
    - $\ell$: 新的输入的长度
    - $d$：embedding dimension
- decoder only
    - KVcache: $K_{past}, V_{past}$
    - 每次新输入时，计算键值（$K_{new}, V_{new}$），时间复杂度为 $O(\ell\cdot d)$，也需要计算 Query $Q_{new}$
    - 计算注意力，
        - $Q=Q_{new}\in \mathbb R^{\ell \cdot d}$
        - $K=[K_{past}, K_{new}]\in \mathbb R^{(L+\ell)\cdot d}$
        - $V=[V_{past}, V_{new}]\in \mathbb R^{(L+\ell)\cdot d}$
        - $A=QK^T\in \mathbb R^{\ell\cdot(\ell+L)}$
            - $q_i$ 要跟 $L+i$ 的 K 计算 score vector；
        - $\text{softmax}(A)\cdot V\in \mathbb R^{\ell\cdot d}$
- 对于 encoder-decoder
    - At every turn, the new input has to be encoded again; for unidirectional attention, only the newly added message needs to be encoded.